<a href="https://colab.research.google.com/github/xo0ol/Train_YOLO_Peanut/blob/main/Train_YOLO_Peanut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# NVIDIA GPU 가용 확인
!nvidia-smi

Sat Aug 23 13:24:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# roboflow 데이터를 unzip 폴더로 압축 해제
!unzip -q /content/peanut_roboflow_data.zip -d /content/peanut_roboflow_data_unzip

In [3]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/peanut_roboflow_data_unzip" --train_pct=0.9

--2025-08-23 13:27:24--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2025-08-23 13:27:25 (52.1 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 812
Number of annotation files: 812
Images moving to train: 730
Images moving to validation: 82


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00


In [6]:
# yaml 생성하기

import yaml
import os

# yaml 파일 생성 함수 만들기
def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # classes.txt 파일에서 class name 가져오기
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # data dictionary 생성
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # yaml 파일 쓰기
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# 경로 설정하기
path_to_classes_txt = '/content/classes.txt'
path_to_data_yaml = '/content/data.yaml'

# 함수 실행
create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 1
names:
- peanut


In [8]:
# 모델 훈련 시작
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=50 imgsz=640 batch=8

Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

In [19]:
# 객체 탐지

# 영상 경로 지정
video_path = r"/content/peanut_test_video.mp4"
output_path = r"/content/peanut_test_video_output.mp4"

# OpenCV로 영상 읽기
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception(f"영상 파일을 열 수 없습니다: {video_path}")

# 영상 정보 가져오기
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 영상 저장용 VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 프레임 단위로 객체 탐지
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지
    results = model.predict(frame)

    # 탐지 결과 그리기
    for result in results:
        annotated_frame = result.plot()  # YOLOv8이 제공하는 plot()으로 바운딩 박스 그림

    # 결과 영상 저장
    out.write(annotated_frame)

cap.release()
out.release()


0: 640x384 1 peanut, 10.9ms
Speed: 3.3ms preprocess, 10.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 12.2ms
Speed: 3.2ms preprocess, 12.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 12.6ms
Speed: 3.6ms preprocess, 12.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 13.3ms
Speed: 3.2ms preprocess, 13.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 12.9ms
Speed: 3.4ms preprocess, 12.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 12.0ms
Speed: 3.3ms preprocess, 12.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 12.8ms
Speed: 3.2ms preprocess, 12.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 peanut, 11.3ms
Speed: 3.2ms preprocess, 11.3ms inference, 1.5ms postprocess per image at shape (1, 3, 64